<div style="
    border: 2px solid #4CAF50; 
    padding: 15px; 
    background-color: #f4f4f4; 
    border-radius: 10px; 
    align-items: center;">

<h1 style="margin: 0; color: #4CAF50;">Neural Networks: Ein Beispiel (Klassifikation) (Lösung)</h1>
<h2 style="margin: 5px 0; color: #555;">DSAI</h2>
<h3 style="margin: 5px 0; color: #555;">Jakob Eggl</h3>

<div style="flex-shrink: 0;">
    <img src="https://www.htl-grieskirchen.at/wp/wp-content/uploads/2022/11/logo_bildschirm-1024x503.png" alt="Logo" style="width: 250px; height: auto;"/>
</div>
<p1> © 2025/26 Jakob Eggl. Nutzung oder Verbreitung nur mit ausdrücklicher Genehmigung des Autors.</p1>
</div>
<div style="flex: 1;">
</div>   

Wir wollen nun auch ein neuronales Netzwerk für die Klassifizierung bauen. Dabei wollen wir ein sehr bekanntes Dataset verwenden (MNIST). Es gibt es in vielen Variationen (zum Beispiel auch mit Kleidung (Fashion-MNIST)) und ist gratis. 

Zuerst wollen wir das normale MNIST Dataset verwenden. Es beinhaltet die handgeschriebenen Zahlen von $0$ bis $9$. Ziel ist es die richtige Zahl zu erkennen.

![MNIST](../resources/MNIST.png)

(von https://de.wikipedia.org/wiki/MNIST-Datenbank)

Insgesamt hat das MNIST Dataset $60\mathrm k$ Trainingsbilder und $10\mathrm k$ Testbilder. Die Klassen sind dabei ziemlich gleichverteilt, sprich es gibt in etwa gleich viele Bilder mit Label "1", Label "2", usw.

# Lösung

Zu Beginn wollen wir sicherstellen, dass jede und jeder das MNIST Dataset heruntergeladen hat. Der Pfad der folgenden Methode kann, wenn nötig, angepasst werden.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
data_path = os.path.join("..", "..", "_data", "mnist_data")

train_dataset = datasets.MNIST(root=data_path, train=True, download=True, transform=transforms.ToTensor()) # ToTensor makes images [0, 1] instead of {1,2,...,255}
test_dataset  = datasets.MNIST(root=data_path, train=False, download=True, transform=transforms.ToTensor())

test_size = len(test_dataset) // 2
valid_size = len(test_dataset) - test_size

test_dataset, valid_dataset = random_split(test_dataset, [test_size, valid_size])

Mit der obigen Methode haben wir direkt ein Torch Dataset erhalten und müssen nur mehr später den Dataloader erstellen.

Kurze **Wiederholung**: *Wie erstellt man sein eigenes Dataset*?

Zum Beispiel so:

In [ ]:
class MyDataSetThatIsNeverUsed(Dataset): 
    def __init__(self, transform=None):
        super().__init__()
        self.transform = transform

    def __len__(self):
        return 0

    def __getitem__(self, idx):
        # here is place for the transformation. Returns input and label
        return torch.tensor([]), torch.tensor(0)

Ansonsten starten wir wieder mit dem device (Prinzipiell eine gute Gewohnheit, dies einmalig am Anfang zu definieren).

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(device)

Nachdem wir die Datasets schon haben, wollen wir nun die Dataloader definieren.

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

Wir wollen uns nun auch noch ein paar Bilder aus dem Trainingsset ansehen.

In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

plt.figure(figsize=(8, 3))
for i in range(6):
    plt.subplot(1, 6, i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title(f"{example_targets[i]}")
    plt.xticks([])
    plt.yticks([])
plt.show()

Als nächstes definieren wir uns das Netzwerk. Auf was müssen wir nun acht geben im Vergleich zur Regression?

In [ ]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(), # Very important! Why? -> We will see that for CNN's we don't need this flattening!
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )
    def forward(self, x):
        return self.layers(x)

In [ ]:
model = MNISTClassifier().to(device)

Welchen Loss wollen wir verwenden? Welchen Optimizer?

In [ ]:
lr = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

Kommen wir nun zur Trainingsmethode. Wir machen diese dieses Mal als eigene Methode. Ebenso machen wir das mit der Evaluierungsmethode. (Grund für die umgekehrte Reihenfolge ist, weil die Trainingsmethode eine Evaluierungsmethode beinhaltet.)

In [ ]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    loss_total = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            loss_total += loss.item() * data.size(0)
            
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    avg_loss = loss_total / total
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, save_path:str=None,
                epochs=20, validate_at=1, print_at=100, patience=3):
    
    if save_path is None:
        save_path = os.path.join("..", "models", "nn_8_best_model.pth")

    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if (batch_idx+1) % print_at == 0:
                print(f"Epoch [{epoch}/{epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        if epoch % validate_at == 0:
            val_loss, val_acc = evaluate_model(model, valid_loader, criterion)
            print(f"Epoch [{epoch}/{epochs}] - Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                torch.save(model.state_dict(), save_path)
                print(f">>> Found a better model and saved it at '{save_path}'")
            else:
                patience_counter += 1
                print(f"No Improvement. Early Stopping Counter: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print("Early Stopping triggered.")
                    break

Last but not least wollen wir nun das Modell trainieren. Dazu definieren wir uns die Hyperparameter zuerst (manche sind der Form halber jetzt doppelt).

In [ ]:
### HYPERPARAMETER ###

model = MNISTClassifier().to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
epochs = 20
validate_at = 1
print_at = 200
early_stopping_patience = 3
save_path = os.path.join("..", "models", "nn_8_best_model_mnist.pth")

In [ ]:
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=epochs, validate_at=validate_at, print_at=print_at, patience=early_stopping_patience, save_path=save_path)

Am Schluss evaluieren wir noch das beste Modell:

In [ ]:
model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate_model(model, test_loader, criterion)
print(f"Finaler Test Loss: {test_loss:.4f}")
print(f"Finale Test Accuracy: {test_acc:.2f}%")

___

Sind wir zufrieden? Was könnte man verbessern?

Man könnte eine (andere) Transformation verwenden.

Berechnen wir dazu mal den Mean und die Varianz (bzw. Standardabweichung der Trainingsdaten)

In [ ]:
mean = 0.
std = 0.
for imgs, _ in train_loader:
    mean += imgs.mean()
    std += imgs.std()

mean /= len(train_loader)
std /= len(train_loader)
print(mean.item(), std.item())

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomRotation(10),    # small data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
data_path = os.path.join("..", "..", "_data", "mnist_data")

train_dataset = datasets.MNIST(root=data_path, train=True, download=True, transform=train_transform) # ToTensor makes images [0, 1] instead of {1,2,...,255}
test_dataset  = datasets.MNIST(root=data_path, train=False, download=True, transform=test_transform)

test_size = len(test_dataset) // 2
valid_size = len(test_dataset) - test_size

test_dataset, valid_dataset = random_split(test_dataset, [test_size, valid_size])

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
### HYPERPARAMETER ###

model = MNISTClassifier().to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
epochs = 10
validate_at = 1
print_at = 200
early_stopping_patience = 3
save_path = os.path.join("..", "models", "nn_8_best_model_mnist_transform.pth")

In [ ]:
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=epochs, validate_at=validate_at, print_at=print_at, patience=early_stopping_patience, save_path=save_path)

In [ ]:
model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate_model(model, test_loader, criterion)
print(f"Finaler Test Loss: {test_loss:.4f}")
print(f"Finale Test Accuracy: {test_acc:.2f}%")

Man könnte nun natürlich auch noch weitere Epochen, ein noch größeres Netzwerk, andere Learning Rate, anderer Optimierer etc. verwenden. Wir sehen aber davon ab.

___

Wir verwenden jetzt das **Fashion-MNIST** Dataset und führen alles nochmal aus.

Es besteht nun aus Kleidungsstücken und dazugehörig 10 Labels. Wir müssen also unser Modell in erster Linie nicht anpassen. 

Auch hier gibt es $60\, \mathrm k$ Trainingsbilder und $10\, \mathrm k$ Testbilder.

Wir kopieren nun die wichtigsten Dinge und ändern sie leicht ab.

In [ ]:
data_path = os.path.join("..", "..", "_data", "fashion_mnist_data")

train_dataset = datasets.FashionMNIST(root=data_path, train=True, download=True, transform=transforms.ToTensor()) # ToTensor makes images [0, 1] instead of {1,2,...,255}
test_dataset  = datasets.FashionMNIST(root=data_path, train=False, download=True, transform=transforms.ToTensor())

test_size = len(test_dataset) // 2
valid_size = len(test_dataset) - test_size

test_dataset, valid_dataset = random_split(test_dataset, [test_size, valid_size])

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

label_dict = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot"
}

plt.figure(figsize=(8, 3))
for i in range(6):
    plt.subplot(1, 6, i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title(f"{label_dict[example_targets[i].item()]}")
    plt.xticks([])
    plt.yticks([])
plt.show()

Wir verwenden nun das gleiche Modell wie vorher, ändern aber den Klassennamen.

In [ ]:
class FashionMNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(), # Very important! Why?
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )
    def forward(self, x):
        return self.layers(x)

Nun trainieren wir das Modell.

In [ ]:
### HYPERPARAMETER ###

model = FashionMNISTClassifier().to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
epochs = 20
validate_at = 1
print_at = 200
early_stopping_patience = 3
save_path = os.path.join("..", "models", "nn_8_best_model_fashion_mnist.pth")

In [ ]:
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=epochs, validate_at=validate_at, print_at=print_at, patience=early_stopping_patience, save_path=save_path)

In [ ]:
model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate_model(model, test_loader, criterion)
print(f"Finaler Test Loss: {test_loss:.4f}")
print(f"Finale Test Accuracy: {test_acc:.2f}%")

Diese Performance ist nicht wirklich gut. Für 10 Klassen bedeutet das, dass wir im Mittel 1 von 10 Klassen falsch zuordnen. Wir betrachten noch kurz die Confusion-Matrix.

In [ ]:
# Confusion Matrix of FashionMNIST model

test_data = test_loader.dataset.dataset.data[test_loader.dataset.indices]
test_targets = test_loader.dataset.dataset.targets[test_loader.dataset.indices]

pred = model(test_data.unsqueeze(1).float().to(device))
cm = confusion_matrix(test_targets.cpu(), pred.argmax(dim=1).cpu())

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_dict.values(), yticklabels=label_dict.values())
plt.title("Confusion Matrix for FashionMNIST Classification")
plt.show()

(zur Erinnerung, $y$-Achse entspricht der Ground-Truth und $x$-Achse entspricht der Vorhersage)

---

## Reicht also immer ein Fully-Connected Neuronal Netzwerk aus?

**Nein!**

Es gibt viele Probleme, die andere Architekturen erwarten. Auch, wenn man in gewissen Situationen vielleicht mit so einer Performance zufrieden ist, werden wir, insbesondere, wenn wir uns später zum Beispiel der **Image-Inpainting** Challenge widmen, sehen, dass wir andere Architekturen brauchen, da diese viel besser funktionieren.

Als nächstes werden wir also eine neue Architektur kennenlernen, welche mit Bildern noch viel besser umgehen kann, als Feed-Forward Neuronal Netzwerke. Die Rede ist von sogenannten ***CNN's*** (*Convolutional Neuronal Networks*).